# The files have been generated in Databricks and provided on the S3 bucket

In [1]:
!pip install boto3

In [2]:
import boto3

session = boto3.Session() # Grabs session details directly from aws configuration in EC2 instance running the Notebook server
session

Session(region_name='eu-central-1')

In [3]:
# Open S3 bucket
s3 = boto3.resource('s3')

bucket_name = 'sjf-project'

bucket = s3.Bucket(bucket_name)
response = f'Bucket \'{bucket_name}\' is accessable.'
print(response)

Bucket 'sjf-project' is accessable.


In [8]:
# Use a previously-built, AWS XGBoost model for training

!pip install sagemaker

import sagemaker

from sagemaker.amazon.amazon_estimator import get_image_uri
# container = get_image_uri(***insert correct arguments here***) 

container = get_image_uri(region_name=boto3.Session().region_name,
                          repo_name='xgboost',
                          repo_version='1.0-1')

The method get_image_uri has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [9]:
# Create pointers to the S3 train and test datasets

s3_path = f"s3://{bucket_name}/"


from sagemaker.session import s3_input

s3_input_train = sagemaker.session.s3_input(s3_data=f"{s3_path}train_sample.csv", content_type="text/csv")
s3_input_valid = sagemaker.session.s3_input(s3_data=f"{s3_path}valid_sample.csv", content_type="text/csv")
#s3_input_test = sagemaker.session.s3_input(s3_data=f"{s3_path}test.csv", content_type="text/csv")

The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
The class sagemaker.session.s3_input has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [10]:
# Create a SageMaker Session

# INSERT CODE HERE
##From LiveCoding

sagemaker_session = sagemaker.Session(boto_session=session)

#The following IAM role ARN was taken from the flutz notebook listed in the SageMaker console:
sm_execution_role = 'arn:aws:iam::898627427171:role/service-role/AmazonSageMaker-ExecutionRole-20201106T104926'


In [11]:
# Create an XGBoost Estimator

# INSERT CODE HERE

output_path = s3_path + "estimator"


# From https://docs.aws.amazon.com/sagemaker/latest/dg/sagemaker-algo-docker-registry-paths.html 
# XGBoost ecr path is: 492215442770.dkr.ecr.eu-central-1.amazonaws.com
ecr_path = '492215442770.dkr.ecr.eu-central-1.amazonaws.com'


xgboost = sagemaker.estimator.Estimator(f"{ecr_path}/sagemaker-xgboost:1.0-1-cpu-py3",
                              sm_execution_role,
                              train_instance_count=1,
                              train_instance_type="ml.m5.large",
                              output_path=output_path,
                              sagemaker_session=sagemaker_session,
                              base_job_name="sjf-xgboost")


train_instance_count has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.
train_instance_type has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


In [12]:
# Select the your specific hyperparameters (Optional)

# INSERT CODE HERE

# From https://docs.aws.amazon.com/sagemaker/latest/dg/xgboost_hyperparameters.html

xgboost.set_hyperparameters(
    eta=0.3,                         # default 0.3 range 0 - 1
    num_round=30,                    # required | valid values: int | The number of rounds to run the training.
    objective = 'reg:squarederror',  # Examples: reg:logistic, multi:softmax, reg:squarederror [default] 
    )


In [14]:
# Fit the model

# INSERT CODE HERE
xgboost.fit({"train": s3_input_train})


2020-11-17 10:45:53 Starting - Starting the training job...
2020-11-17 10:45:55 Starting - Launching requested ML instances......
2020-11-17 10:47:01 Starting - Preparing the instances for training...
2020-11-17 10:47:47 Downloading - Downloading input data...
2020-11-17 10:48:14 Training - Downloading the training image...
2020-11-17 10:48:52 Uploading - Uploading generated training model
2020-11-17 10:48:52 Completed - Training job completed
INFO:sagemaker-containers:Imported framework sagemaker_xgboost_container.training
INFO:sagemaker-containers:Failed to parse hyperparameter objective value reg:squarederror to Json.
Returning the value itself
INFO:sagemaker-containers:No GPUs detected (normal if no gpus installed)
INFO:sagemaker_xgboost_container.training:Running XGBoost Sagemaker in algorithm mode
INFO:root:Determined delimiter of CSV input is ','
INFO:root:Determined delimiter of CSV input is ','
[10:48:41] 1037x32 matrix with 33184 entries loaded from /opt/ml/input/data/train?f

In [17]:
# Deploy your model to an endpoint to perform predictions
xgb_predictor = xgboost.deploy(
    initial_instance_count = 1, 
    instance_type = 'ml.t2.medium')     #'ml.m4.xlarge') | 'ml.t2.medium'
    

-----------------!

In [21]:
# Configure the predictor's serializer and deserializer

from sagemaker.predictor import csv_serializer, csv_deserializer

xgb_predictor.serializer = csv_serializer
xgb_predictor.deserializer = csv_deserializer
#xgb_predictor.content_type = "text/csv"

In [27]:
xgb_predictor.endpoint

The endpoint attribute has been renamed in sagemaker>=2.
See: https://sagemaker.readthedocs.io/en/stable/v2.html for details.


'sjf-xgboost-2020-11-17-10-58-57-102'

# Predictions

In [28]:
s3_input_valid.head()

AttributeError: 'DeprecatedClass' object has no attribute 'head'

In [25]:
#prediction = xgb_predictor.predict(s3_input_valid.iloc[0, 1:])
#prediction

AttributeError: 'DeprecatedClass' object has no attribute 'iloc'